In [1]:
!pip install pandas
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Isi nama dan NIM seluruh anggota kelompok di sini
#   Nama    NIM
#   Dzaky Raihan Ahmad     1301204316
#   Auzan Fadhlan     1301204020
#   Zahran Raihansyah Likumahua 1301204049

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree


# Load Dataset
Inisialisasi variable, masukan NIM di sini untuk random state.

Gunakan NIM salah satu anggota kelompok.



In [3]:
filepath = "./seeds_dataset.csv"

# Isi nim anda di sini
nim = 1301204316

Load dataset, cek konten data

Target kelas: **Category** 

In [5]:
df = pd.read_csv(filepath, sep="\t")
df.sample(10, random_state=nim)

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len,Category
23,12.08,13.23,0.8664,5.099,2.936,1.415,4.961,1
179,12.21,13.47,0.8453,5.357,2.893,1.661,5.178,3
48,14.79,14.52,0.8819,5.545,3.291,2.704,5.111,1
39,14.28,14.17,0.8944,5.397,3.298,6.685,5.001,1
185,11.56,13.31,0.8198,5.363,2.683,4.062,5.182,3
72,17.26,15.73,0.8763,5.978,3.594,4.539,5.791,2
160,12.54,13.67,0.8425,5.451,2.879,3.082,5.491,3
115,19.06,16.45,0.8854,6.416,3.719,2.248,6.163,2
108,19.94,16.92,0.8752,6.675,3.763,3.252,6.550,2
165,12.10,13.15,0.8793,5.105,2.941,2.201,5.056,3


Training/test split **60/40**

In [6]:
# X: semua kolom, Y: kolom terakhir (category)
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6, random_state=nim)

# We perform bootstrap on the train data
# Keep test data for final accuracy testing
len(x_train), len(x_test)

(126, 84)

# Random Forest
Build a Random Forest, consisting of 20 Decision Tree classifer, with the following steps:

1. Prepare Bootstrap data (choose randomly from the *training data*, use sampling with replacement)
2. Choose 2-4 random features/atributes for each tree
3. Build Decision Tree classifier using step (1) and (2)
4. Calculate OOB score within the tree only

**NOTE**
This whole exercise has been simplified.
- Tree can be created using a library, therefore you may not be able control the feature selection on node level
- Random features are only selected in the beginning of each tree.
- OOB score is only calculated within its own tree instead.
- Typically Random Forest don't use train/test split and can be validated using OOB error. We simplify this to make it easier to test.
- You are not allowed to use a Random Forest or Bagging library.


In [7]:
np.random.seed(nim)
n_tree = 20

forest = []
bootstrap_columns = []
for i in range(n_tree):
    # 1. Bootstrapping
    # select random rows
    rows = np.random.choice(x_train.index, size=len(x_train), replace=True)
    x_bootstrap = x_train.loc[rows]
    y_bootstrap = y_train.loc[rows]

    # Out of Bag data
    non_selected_rows = np.setdiff1d(x_train.index, rows)
    x_oob = x_train.loc[non_selected_rows]
    y_oob = y_train.loc[non_selected_rows]

    # 2. Select random features (make sure to keep track the selected columns)
    # Check for any repeated combinations
    feature_taken = np.random.randint(2, 4)
    cols = np.random.choice(x_bootstrap.columns, size=feature_taken, replace=True)
    bootstrap_columns.append(cols)

    # filter based on the selected columns
    x_bootstrap = x_bootstrap.loc[:, cols]

    # 3. Build decision tree
    # Let it grow, no pruning
    tree_model = tree.DecisionTreeClassifier()
    tree_model.fit(x_bootstrap, y_bootstrap)

    # Add it to the forest
    forest.append(tree_model)

    # 4. Calculate oob score only within its own tree
    oob_predict = tree_model.predict(x_oob.loc[:, cols])
    oob_score   = tree_model.score(x_oob.loc[:, cols], y_oob)
    print(f"Tree {i}, {len(x_oob)} data, oob score: {oob_score}")



Tree 0, 44 data, oob score: 0.8409090909090909
Tree 1, 47 data, oob score: 0.8936170212765957
Tree 2, 45 data, oob score: 0.8
Tree 3, 49 data, oob score: 0.8163265306122449
Tree 4, 51 data, oob score: 0.7254901960784313
Tree 5, 47 data, oob score: 0.8723404255319149
Tree 6, 45 data, oob score: 0.8222222222222222
Tree 7, 46 data, oob score: 0.782608695652174
Tree 8, 43 data, oob score: 0.9302325581395349
Tree 9, 41 data, oob score: 0.9512195121951219
Tree 10, 44 data, oob score: 0.7727272727272727
Tree 11, 55 data, oob score: 0.8181818181818182
Tree 12, 48 data, oob score: 0.8333333333333334
Tree 13, 48 data, oob score: 0.7916666666666666
Tree 14, 45 data, oob score: 0.7555555555555555
Tree 15, 44 data, oob score: 0.9090909090909091
Tree 16, 42 data, oob score: 0.8095238095238095
Tree 17, 49 data, oob score: 0.7142857142857143
Tree 18, 46 data, oob score: 0.9347826086956522
Tree 19, 43 data, oob score: 0.8604651162790697


# Prediction and Voting
Run prediction on x_test data for each tree

Also calculate accuracy per tree here

In [13]:
all_preds = []
for i, tree_model in enumerate(forest):
    # Each tree has different column requirements
    # Make sure you use the correct columns for each tree
    x_test_filtered = x_test.loc[:, bootstrap_columns[i]]

    predictions = tree_model.predict(x_test_filtered)

    # Might as well calculate accuracy per tree here...

    acc = tree_model.score(x_test_filtered, y_test)
    print(acc)

    # add it to all_preds for voting later
    all_preds.append(predictions)

all_preds = np.array(all_preds)

0.7619047619047619
0.8809523809523809
0.7738095238095238
0.7976190476190477
0.8571428571428571
0.9285714285714286
0.8333333333333334
0.8214285714285714
0.8452380952380952
0.9404761904761905
0.8452380952380952
0.7976190476190477
0.8214285714285714
0.7738095238095238
0.7976190476190477
0.8809523809523809
0.8571428571428571
0.7857142857142857
0.8928571428571429
0.8095238095238095


In [14]:
for _ in bootstrap_columns:
    print(*_, sep=", ")

Compactness, Compactness, Kernel_width
Area, Groove_len
Kernel_width, Area, Compactness
Assymetry, Kernel_len
Kernel_len, Kernel_len, Area
Groove_len, Area, Kernel_width
Kernel_width, Perimeter, Kernel_len
Kernel_len, Perimeter
Kernel_width, Area, Assymetry
Groove_len, Area, Compactness
Groove_len, Compactness
Kernel_width, Kernel_width, Perimeter
Kernel_len, Perimeter
Kernel_width, Compactness
Assymetry, Groove_len
Kernel_len, Groove_len
Groove_len, Kernel_len
Perimeter, Kernel_len
Groove_len, Kernel_width, Area
Perimeter, Kernel_len


### Perform voting on the predictions from the Random Forest

In [15]:
# Insert voting code here
voted_predictions = np.array([np.bincount(pred).argmax() for pred in all_preds.T])
print(voted_predictions)

# Calculate accuracy of Test data on the Forest
acc = np.sum(voted_predictions == y_test) / len(y_test) *100
print(acc, "%")



[3 3 1 1 3 2 3 2 2 3 1 1 3 2 1 2 2 1 2 1 1 1 1 2 1 2 1 2 3 2 1 3 2 3 3 2 2
 3 3 3 3 1 1 3 3 2 1 2 3 1 2 2 3 3 2 1 2 1 1 3 2 2 2 1 2 3 1 3 1 3 1 1 3 3
 2 2 1 3 3 2 2 1 1 2]
88.09523809523809 %


After you're done with all this, please check the excel file and put the answers for every column there.